In [1]:
import numpy as np
import time
import logging
from module.print import printTable, printNode
from module.node import Node
from module.generate_children import get_all_children
from module.sortNode import sortNode
from module.createBoatActions import boat_actions


def logging_file(open_list, close_list):
    logging.basicConfig(filename='aStar_new.log', level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info('********open list********, len = {}'.format(len(open_list)))
    for open_data in open_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   open_data.state,
                                                                                    open_data.data[0], open_data.data[1], open_data.data[2],
                                                                                    open_data.boatA, open_data.boatB))
    logging.info('********close list********, len = {}'.format(len(close_list)))
    for close_data in close_list:
        logging.debug('{}\n> cost = {}, step = {}, f(n) = {}\n> {}\n> {}'.format(   close_data.state,
                                                                                    close_data.data[0], close_data.data[1], close_data.data[2],
                                                                                    close_data.boatA, close_data.boatB))
    logging.info('--------------------------------------------------------------------------------------------------------')

def calculate_path(curr_node):
    result = []
    result.append(curr_node)
    parent_node = curr_node.parent
    while (parent_node):
        result.append(parent_node)
        parent_node = parent_node.parent
    return result

def uniform_cost_search(goal, start_node, setting, boatSetting):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 0
    
    while True:
        if not open_list:
            print ("no solution!!!")
            break
        
        curr_node = open_list.pop()

        if (curr_node.state['m'] == goal[0] and curr_node.state['c'] == goal[1] ):
            #printNode(curr_node, setting)
            result = calculate_path(curr_node)
            print("****************")
            printTable(result, setting)
            break

        close_list.append(curr_node)
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar) # cost: 0 , time: 1
        
        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0
            
            for i in range(len(close_list)):
                if np.array_equal(child.state, close_list[i].state): #and np.array_equal(child.data, close_list[i].data):
                    sameNode_in_close_list = 1
                    break
            
            for j in range(len(open_list)):
                if np.array_equal(child.state, open_list[j].state):
                    sameNode_in_open_list = 1
                    if child.data[ setting['costOrStep'] ] < open_list[j].data[ setting['costOrStep'] ]:
                        open_list.pop(j)
                    break
            
            if (sameNode_in_close_list != 1 and sameNode_in_open_list != 1):
                open_list.append(child)
        sortNode(open_list, setting['costOrStep'], useAStar)
        logging_file(open_list, close_list)

def AStart_algorithm(goal, start_node, setting, boatSetting):
    open_list = []
    close_list = []
    open_list.append(start_node)
    useAStar = 1
    
    while True:
        if not open_list:
            print ("no solution!!!")
            break
        
        curr_node = open_list.pop()

        if (curr_node.state['m'] == goal[0] and curr_node.state['c'] == goal[1] ):
            printTable(curr_node, setting)
            #result = calculate_path(curr_node)
            #printTable(result, setting)
            break

        close_list.append(curr_node)
        children = get_all_children(curr_node, boatA_operations, boatB_operations, setting, boatSetting, useAStar) 
        
        for child in children:
            sameNode_in_open_list = 0
            sameNode_in_close_list = 0

            for i in range(len(open_list)):
                if np.array_equal(child.state, open_list[i].state):
                    if child.data[2] < open_list[i].data[2]: # beter => remove older, append newer, AStar_f
                        open_list.pop(i)
                    else : # newer no better
                        sameNode_in_open_list = 1
                    break

            if (sameNode_in_open_list != 1):
                for j in range(len(close_list)):
                    if np.array_equal(child.state, close_list[j].state):
                        if child.data[2] < close_list[j].data[2]: # beter => remove older, append newer, AStar_f
                            close_list.pop(j)
                        else : # newer no better
                            sameNode_in_close_list = 1
                        break

                if (sameNode_in_close_list != 1):
                    open_list.append(child)
        sortNode(open_list, setting['costOrStep'], useAStar)
        logging_file(open_list, close_list)


totalM = int(input("請輸入傳教士的人數: ")) # total Missionaries (right bank)  
totalC = int(input("請輸入野人的人數: "))  # total cannibals (right bank) 
while totalM < totalC and totalM != 0 or  totalM < 0 or totalC < 0:
    totalM = int(input("請再次輸入傳教士的人數: ")) # total Missionaries (right bank)  
    totalC = int(input("請再次輸入野人的人數: "))  # total cannibals (right bank) 

costOrStep = int(input("cost輸入0, step輸入1: "))
while costOrStep != 0 and costOrStep != 1:
    costOrStep = int(input("cost輸入0, step輸入1: "))

boatSetting = dict()
boatSetting['A'] = { 'capacity': 2, 'cost': 3,  'step' : 1, 'pos' : 1} # pos : right bank: 1, left bank: -1
boatSetting['B'] = { 'capacity': 3, 'cost': 25, 'step' : 1, 'pos' : 1}
setting = {'totalM': totalC, 'totalC' : totalC, 'costOrStep' : costOrStep}

boatA_operations = boat_actions('A', boatSetting)
boatB_operations = boat_actions('B', boatSetting)

start_node = Node(setting['totalM'], setting['totalC'], 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
goal_node = [0, 0]
start_time = time.time()
uniform_cost_search(goal_node, start_node, setting, boatSetting)
end = time.time()
print("The time of execution of above program is :", end-start_time)
#AStart_algorithm(goal_node, start_node, setting, boatSetting)

#for m in range(3):
#    for n in range (3, 11):

#        setting = {'totalM': m+n, 'totalC' : n, 'costOrStep' : 1}
#        print(m+n, n)
#        start_node = Node(m + n, n, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, None, setting, boatSetting) # 初始狀態節點
#        goal_node = [0, 0]
#        AStart_algorithm (goal_node, start_node, setting, boatSetting)
#        uniform_cost_search(goal_node, start_node, setting, boatSetting)

boatA 有6種動作，各為: [[0, 0, 3, 1], [0, 1, 3, 1], [0, 2, 3, 1], [1, 0, 3, 1], [1, 1, 3, 1], [2, 0, 3, 1]]
boatB 有9種動作，各為: [[0, 0, 25, 1], [0, 1, 25, 1], [0, 2, 25, 1], [0, 3, 25, 1], [1, 0, 25, 1], [1, 1, 25, 1], [2, 0, 25, 1], [2, 1, 25, 1], [3, 0, 25, 1]]
****************
The time of execution of above program is : 0.20474910736083984
